In [ ]:
import os
import json
import time
from pathlib import Path

import pandas as pd
import google.generativeai as genai
from tqdm.notebook import tqdm # Sử dụng tqdm.notebook để thanh progress hiển thị đẹp hơn trong notebook

In [ ]:
GOOGLE_API_KEY = ""

# Tên model của Gemini dùng để đánh giá (Judge)
JUDGE_MODEL_NAME = "gemini-2.5-flash"

# Đường dẫn đến file rubric JSON chứa các tiêu chí
RUBRIC_PATH = "/content/prompt_judge_minh.json"

# Đường dẫn đến file dữ liệu .jsonl cần được đánh giá
INPUT_DATA_PATH = "/content/samples_detailed_prompt_context_2025-09-10T17-07-50.668664.jsonl"

In [ ]:
# Cell 3: Định nghĩa lớp LLMJudge (Phiên bản cập nhật xử lý lỗi API)

class LLMJudge:
    def __init__(self, rubric_path: str, api_key: str, judge_model: str):
        if not api_key:
            raise ValueError("Không tìm thấy GOOGLE_API_KEY. Vui lòng thiết lập trong Cell 2.")

        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(judge_model)
        self.metrics = self._load_rubric(rubric_path)
        self.metric_names = [m['name'] for m in self.metrics]

        print(f"✅ LLM Judge đã được khởi tạo với model: {judge_model}")
        print(f"✅ Đã tải thành công {len(self.metrics)} tiêu chí từ: {rubric_path}")

    def _load_rubric(self, path: str) -> list:
        """Tải rubric từ file JSON."""
        with open(path, 'r', encoding='utf-8') as f:
            return json.load(f)

    def _create_bulk_prompt(self, data_doc: dict, model_response: str) -> str:
        """Tạo một prompt duy nhất để đánh giá TẤT CẢ các tiêu chí cùng lúc."""
        criteria_text = "\n".join([f'- **{metric["name"]}**: {metric["description"]}' for metric in self.metrics])

        return f"""
Bạn là một chuyên gia đánh giá AI công tâm và chính xác. Nhiệm vụ của bạn là đánh giá câu trả lời của một mô hình ngôn ngữ dựa trên một bộ tiêu chí được cung cấp.

--- NGỮ CẢNH ---
{data_doc.get('context', 'Không có')}

--- CÂU HỎI ---
{data_doc.get('question', 'Không có')}

--- CÁC Lựa CHỌN ---
{data_doc.get('options', 'Không có')}

--- CÂU TRẢ LỜI CỦA MODEL CẦN ĐÁNH GIÁ ---
{model_response}

--- BỘ TIÊU CHÍ ĐÁNH GIÁ ---
Hãy cho điểm câu trả lời trên theo thang điểm từ 1 đến 5 cho mỗi tiêu chí sau:
{criteria_text}

--- NHIỆM VỤ CỦA BẠN ---
Hãy đánh giá câu trả lời của model theo TẤT CẢ các tiêu chí đã cho.
Câu trả lời của bạn BẮT BUỘC phải là một đối tượng JSON duy nhất, không có bất kỳ văn bản giải thích nào trước hoặc sau nó.
Đối tượng JSON phải chứa các key tương ứng với tên tiêu chí, và value là điểm số (số nguyên từ 1 đến 5).
Ví dụ định dạng output:
{{
  "tính súc tích": 4,
  "tính dễ hiểu": 5,
  "tính chặt chẽ": 3,
  "tính liên quan": 5,
  "tính đúng đắn": 4,
  "tính đầy đủ": 3,
  "Cấu trúc câu trả lời": 4
}}
"""

    def _judge_all_criteria(self, data_doc: dict, model_response: str) -> dict:
        """
        Gửi một yêu cầu duy nhất đến Gemini API để đánh giá tất cả các tiêu chí.
        Bao gồm logic retry nâng cao: nghỉ 30 giây nếu gặp lỗi API/kết nối.
        """
        prompt = self._create_bulk_prompt(data_doc, model_response)
        default_scores = {name: 0 for name in self.metric_names}
        max_retries = 5  # Giới hạn số lần thử lại để tránh lặp vô hạn
        attempt = 0

        while attempt < max_retries:
            try:
                response = self.model.generate_content(prompt)
                cleaned_text = response.text.strip().replace("```json", "").replace("```", "")
                judgement = json.loads(cleaned_text)

                # Kiểm tra xem có đủ key không
                if all(key in judgement for key in self.metric_names):
                    return judgement # Thành công, trả về kết quả
                else:
                    # Lỗi JSON hợp lệ nhưng thiếu key
                    print(f"\nLỗi: JSON trả về thiếu key. Thử lại sau 1 giây...")
                    time.sleep(1)

            except Exception as e:
                error_message = str(e).lower()
                # Kiểm tra các thông điệp lỗi cụ thể
                if "api key not valid" in error_message or "connection aborted" in error_message or "remote end closed" in error_message:
                    attempt += 1
                    print(f"\n🔴 Lỗi API hoặc kết nối. Tạm dừng 30 giây trước khi thử lại (lần {attempt}/{max_retries})...")
                    time.sleep(30)
                else:
                    # Các lỗi khác (ví dụ: JSONDecodeError)
                    attempt += 1
                    print(f"\n🟡 Lỗi không xác định: {e}. Thử lại sau 1 giây (lần {attempt}/{max_retries})...")
                    time.sleep(1)

        print(f"\n❌ Lỗi: Không thể nhận được kết quả hợp lệ từ API sau {max_retries} lần thử.")
        return default_scores

    def process_and_evaluate_file(self, input_path: str) -> pd.DataFrame:
        """Xử lý toàn bộ file đầu vào, đánh giá từng dòng và trả về kết quả dưới dạng DataFrame."""
        all_results = []
        try:
            with open(input_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        except FileNotFoundError:
            print(f"❌ LỖI: Không tìm thấy file đầu vào tại {input_path}")
            return pd.DataFrame()

        for line in tqdm(lines, desc="Đang đánh giá các dòng dữ liệu"):
            if not line.strip():
                continue

            data_record = json.loads(line)
            doc_data = data_record['doc']
            model_response = data_record['resps'][0][0]

            row_data = {
                'context': doc_data.get('context', ''),
                'question': doc_data.get('question', ''),
                'options': str(doc_data.get('options', '')),
                'response': model_response
            }

            scores = self._judge_all_criteria(doc_data, model_response)
            row_data.update(scores)

            valid_scores = [s for s in scores.values() if isinstance(s, (int, float)) and s > 0]
            row_data['average_score'] = sum(valid_scores) / len(valid_scores) if valid_scores else 0

            all_results.append(row_data)

        return pd.DataFrame(all_results)

In [ ]:
# Khởi tạo judge với các cấu hình đã thiết lập
judge = LLMJudge(
    rubric_path=RUBRIC_PATH,
    api_key=GOOGLE_API_KEY,
    judge_model=JUDGE_MODEL_NAME
)

✅ LLM Judge đã được khởi tạo với model: gemini-2.5-flash
✅ Đã tải thành công 7 tiêu chí từ: /content/prompt_judge_minh.json


In [ ]:
print("--- Bắt đầu quy trình đánh giá LLM Judge ---")

# Sử dụng pathlib để xử lý đường dẫn một cách an toàn
input_path = Path(INPUT_DATA_PATH)

# Lấy tên model từ tên thư mục cha của file input
model_name = input_path.parent.name
print(f"Phát hiện model name: {model_name}")

# Chạy đánh giá và nhận về một DataFrame
results_df = judge.process_and_evaluate_file(input_path=str(input_path))

if not results_df.empty:
    # Tạo đường dẫn file output động
    output_dir = Path("output")
    output_dir.mkdir(exist_ok=True) # Tạo thư mục output nếu chưa có
    output_path = output_dir / f"{model_name}_scores.csv"

    # Lưu DataFrame ra file CSV
    print(f"\nĐang lưu kết quả ra file CSV tại: {output_path}")
    results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"🎉 Đã lưu thành công kết quả vào {output_path}")

    # Hiển thị 5 dòng đầu tiên của kết quả
    print("\nXem trước 5 dòng kết quả đầu tiên:")
    display(results_df.head())
else:
    print("Không có kết quả nào để lưu.")

print("\n--- Quy trình đánh giá đã kết thúc ---")

--- Bắt đầu quy trình đánh giá LLM Judge ---
Phát hiện model name: content


Đang đánh giá các dòng dữ liệu:   0%|          | 0/200 [00:00<?, ?it/s]